# NY Parking Violation Data Analysis

**Mini Project**

IISC | M.Tech (Online) | DSBA
DA 231-O Data Engineering at Scale

Mentor: **Yogesh Simmhan** (simmhan@iisc.ac.in)

## Team

1. Satyam Kumar | satyamk@iisc.ac.in | 9092696415
2. Siva Kranthi Kumar Mallipeddi | sivam@iisc.ac.in | 9986778909
3. Sreedhar Reddy Vundela | sreedharv@iisc.ac.in | 9886702749
4. Sudhakar Kulkarni Mukayya | sudhakark@iisc.ac.in | 9035076656

## Problem

* Exploratory data analysis of **NYC Parking violation** data for **2017** – **2021** years (5)
* Infer from the data analysis findings

> **Dataset** : https://data.cityofnewyork.us/City-Government/Parking-Violations-Issued-Fiscal-Year-2022/pvqr-7yc4

## Environment

* Python 3
* Spark 3.0 (colab) / 3.1 (local)
* Spark Data Frames & Spark SQL
* Local & Google Colab

## Dataset Details

Dataset contains New York city Police Department issued parking violations details

### Raw Dataset details
- Fiscal year data (July 1 - June 30) is available. That means one file contains data for 2 years
- All years data has same 43 columns
- Total Number of records: **xxxx** # TODO

### Processed Data details
- Removed 30 columns which are not used in our analysis (Most of the removed columns don't have data for > 50% records)
- Total Columns: 13
- Total Number of records: **xxxx** # TODO

### Columns Summary

| S. No. | Source   Column Name    | Description/Comment                                  |
|--------|-------------------------|------------------------------------------------------|
| 1      | SUMMONS NUMBER          | UNIQUE IDENTIFIER OF SUMMONS                         |
| 2      | PLATE ID                | REGISTERED PLATE ID                                  |
| 3      | REGISTRATION STATE      | STATE OF PLATE   REGISTRATION                        |
| 4      | PLATE TYPE              | TYPE OF PLATE                                        |
| 5      | ISSUE DATE              | ISSUE DATE                                           |
| 6      | VIOLATION CODE          | TYPE OF   VIOLATION                                  |
| 7      | VEHICLE BODY TYPE       | VEHICLE BODY TYPE   WRITTEN ON SUMMONS (SEDAN, ETC.) |
| 8      | VEHICLE MAKE            | MAKE OF CAR WRITTEN ON SUMMONS                       |
| 9      | VIOLATION PRECINCT      | POLICE STATION OF VIOLATION                          |
| 10     | ISSUER PRECINCT         | PRECINCT OF ISSUANCE                                 |
| 11     | VIOLATION TIME          | VIOLATION TIME                                       |
| 12     | VEHICLE COLOR           | CAR COLOR WRITTEN ON   SUMMONS                       |
| 13     | VIOLATION   DESCRIPTION | DESCRIPTION OF   VIOLATION                           |

### Columns Details

#### Plate Type

Registration Class Codes for vehicles. 3 letters code

Common Plate types are 
* Passenger Vehicles (PAS): standard issue plates
* Commercial Vehicles (COM): Full-size vans and most pickups
* Medallion (OMT): Taxis
* Personalized Plates (SRF): cars, mini-vans, SUVs and some pick-ups registered as passenger class
* Special Omnibus Rentals (OMS)

https://dmv.ny.gov/registration/registration-class-codes

#### Violation Code
Type of violation. Codes are from 1-99. Fines are charged based on this

https://data.cityofnewyork.us/api/views/pvqr-7yc4/files/7875fa68-3a29-4825-9dfb-63ef30576f9e?download=true&filename=ParkingViolationCodes_January2020.xlsx

#### Vehicle Body Type

Common Vehicle body types are 
* suburban(SUBN): Vehicle that can be used to carry passengers and cargo
* four-door sedan (4DSD)
* Van Truck (VAN
* Delivery Truck (DELV)
* Pick-up Truck (PICK)
* two-door sedan (2DSD) 
* Sedan (SEDN)

https://nysdmv.custhelp.com/app/answers/detail/a_id/491/kw/body%20type%20subn

#### Vehicle Make

The DMV code for the make of a vehicle that appears on the registration. The DMV make code is the first 5 letters of the vehicle’s make name. If the vehicle make is more than one word, the make code is the first 2 letters of the first two words with a slash in between

Common Vehicle Makes are 
* Honda (HONDA)
* Toyota (TOYOT)
* Ford (FORD)
* Nissan (NISSA)
* Chevrolet (CHEVR)
* mercedes benz (ME/BE)

https://data.ny.gov/Transportation/Vehicle-Makes-and-Body-Types-Most-Popular-in-New-Y/3pxy-wy2i
https://data.ny.gov/api/assets/83055271-29A6-4ED4-9374-E159F30DB5AE

#### Vehicle Colors

Common colors are
* Gray (GY)
* White (WH)
* Black (BK)
* Blue (BL)
* Red (RD)

## Config

In [18]:
# Execution environment. Use False for local execution
colab_env = False

# Use Sample file for speedy execution
sample_file = True
sample_file_path = 'TODO:' if colab_env else "../data/sample-100000.csv"

# Specify the raw CSV files path
raw_CSV_path = 'TODO:' if colab_env else "../data/Parking*.csv"

csv_files = sample_file_path if sample_file else raw_CSV_path

# For faster execution. Some statements are skipped based on this check
presenting = True

# Specify the years for which we are reading the data from CSV
years = [2017, 2018, 2019, 2020, 2021]

# Schema Types. Only specify for the non-string type & NULL columns. Others  are considered as string
schema_types = {
  "Summons Number": {"type": "long", "null": False},
  "Issue Date"    : {"type": "date" if sample_file else "string", "null": True}, 
  "Violation Code": {"type": "integer", "null": True},
  "Violation Precinct": {"type": "integer", "null": True},
  "Issuer Precinct": {"type": "integer", "null": True},
}

# Columns which are used in the analysis. Other columns will be removed
used_columns = ["Summons Number", "Plate ID", "Registration State", "Plate Type", "Issue Date", "Violation Code", "Vehicle Body Type", "Vehicle Make", "Violation Precinct", "Issuer Precinct", "Violation Time", "Vehicle Color", "Violation Description"]

# All the columns which are there in the datset (Need to be in CSV file order)
schema_columns = ["Summons Number", "Plate ID", "Registration State", "Plate Type", "Issue Date", "Violation Code", "Vehicle Body Type", "Vehicle Make", "Issuing Agency", "Street Code1", "Street Code2", "Street Code3", "Vehicle Expiration Date", "Violation Location", "Violation Precinct", "Issuer Precinct", "Issuer Code", "Issuer Command", "Issuer Squad", "Violation Time", "Time First Observed", "Violation County", "Violation In Front Of Or Opposite", "House Number", "Street Name", "Intersecting Street", "Date First Observed", "Law Section", "Sub Division", "Violation Legal Code", "Days Parking In Effect    ", "From Hours In Effect", "To Hours In Effect", "Vehicle Color", "Unregistered Vehicle?", "Vehicle Year", "Meter Number", "Feet From Curb", "Violation Post Code", "Violation Description", "No Standing or Stopping Violation", "Hydrant Violation", "Double Parking Violation"] if not sample_file else [i.lower().replace(" ", '_') for i in used_columns]

# Generates the sample CSV 
if not sample_file:
  sample_CSV_generate = True # Generate the sample CSV
  sample_CSV_records = 100000 # No. of records to write into the sample CSV file
  sample_CSV_path = f"../data/sample-{sample_CSV_records}.csv" # path to save
  sample_seed = sample_CSV_records # Seed value so that we get same random records

## Spark Setup

In [19]:
# For google Colab
if colab_env:
  !apt-get install openjdk-8-jdk-headless -qq > /dev/null
  !wget -q https://mirrors.estointernet.in/apache/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz -P /content/
  !tar xf /content/drive/Shareddrives/DA231-2021-Aug-Public/spark-3.0.3-bin-hadoop2.7.tgz
  !pip install -q findspark

  import os
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

## Imports

In [20]:
# Reload all changed modules every time before executing a new line
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
# PySpark related
import findspark
findspark.init()
findspark.find()

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col # Frequently using this. hence imported separately

# user defined modules import
import data_preprocess as dp
import basic_analysis as ba
import precinct_based_analysis as pba
import time_based_analysis as tba
import season_based_analysis as sba
import revenue_based_analysis as rba
import mis_analysis as ma

## Spark Session

In [22]:
spark = (
          SparkSession
            .builder
            .master("local[*]") # Using all available cores
            .appName("NY Parking Violation Analysis")
            .getOrCreate()
        )
spark

## Data Pre-processing

### Reading CSV files into DataFrame

In [23]:
# Better performance than inferSchema=True
NY_schema = dp.get_schema(schema_columns, schema_types)

org_df = spark.read.option("header", True).schema(NY_schema).csv(csv_files)

In [24]:
if not sample_file:
  print(f'Shape : {(org_df.count(), len(org_df.columns))}')
  org_df.printSchema()
  org_df.show(2)

In [25]:
if not sample_file:
  presenting and org_df.summary().toPandas().transpose() # More execution time

### Pre-processing

In [26]:
# No pre-processing while using Sample file
df = org_df
if not sample_file:
  df = dp.remove_unused_columns(df, used_columns)
  df = dp.drop_duplicates(df)
  df = dp.santize_column_names(df)
  dp.assert_uniqueness(df, column_name="summons_number")
  df = dp.convert_to_date(df, column_name="issue_date", format="MM/dd/yyyy")
  df = dp.remove_outside_years_data(df, years, "issue_date")
  # df = dp.remove_invalid_violation_code_data(df) # TODO: Maybe we can handle in individual file

## Sample file generation

In [65]:
from pathlib import Path
from shutil import rmtree

def write_CSV(df, CSV_path):
  
  # Creates CSV in a folder. But memory efficient
  df.coalesce(1).write.mode("overwrite").csv(CSV_path, header=True) 

  # OOM
  # df.toPandas().to_csv(CSV_path, index=False)

  # Moving file to data folder
  f_path = list(Path(CSV_path).glob('*.csv'))[0]
  Path(f_path).rename(CSV_path+'.tmp')
  rmtree(CSV_path)
  Path(CSV_path+'.tmp').rename(CSV_path)


In [66]:
if (not sample_file and sample_CSV_generate):
  total_records = df.count()
  fraction = (sample_CSV_records+10000)/total_records # Exact records are not coming. Hence increasing the fraction using 10k
  sample_df = df.sample(fraction=fraction, seed=sample_seed).limit(sample_CSV_records) # Getting exact number of records
  print(f"Sample records: {sample_df.count()}")
  write_CSV(sample_df, sample_CSV_path)
  df = sample_df

## Final DF for Analysis

In [27]:
total_records = df.count()
print(f'Shape : {(total_records, len(df.columns))}')
df.printSchema()
df.show(2)

Shape : (100000, 13)
root
 |-- summons_number: string (nullable = true)
 |-- plate_id: string (nullable = true)
 |-- registration_state: string (nullable = true)
 |-- plate_type: string (nullable = true)
 |-- issue_date: string (nullable = true)
 |-- violation_code: string (nullable = true)
 |-- vehicle_body_type: string (nullable = true)
 |-- vehicle_make: string (nullable = true)
 |-- violation_precinct: string (nullable = true)
 |-- issuer_precinct: string (nullable = true)
 |-- violation_time: string (nullable = true)
 |-- vehicle_color: string (nullable = true)
 |-- violation_description: string (nullable = true)

+--------------+--------+------------------+----------+----------+--------------+-----------------+------------+------------------+---------------+--------------+-------------+---------------------+
|summons_number|plate_id|registration_state|plate_type|issue_date|violation_code|vehicle_body_type|vehicle_make|violation_precinct|issuer_precinct|violation_time|vehicle_colo

In [28]:
presenting and df.describe().toPandas().transpose() # More execution time

,0,1,2,3,4
summary,count,mean,stddev,min,max
summons_number,100000,6.7570832783323E9,2.306970432979163E9,1263367471,8999603295
plate_id,100000,Infinity,NaN,0001KRD,ZZZ510Y
registration_state,100000,99.0,0.0,99,WY
plate_type,100000,999.0,0.0,999,VAS
issue_date,100000,None,None,2017-07-23,2021-09-29
violation_code,100000,33.32032,17.93011907348359,0,99
vehicle_body_type,99814,3.8974668275030155,0.513447992485732,00,YY
vehicle_make,99951,None,None,ACL,ZHNG
violation_precinct,100000,28.4088,38.13138133588172,0,94


In [46]:
print('Top 5 Violation precicts')
print(pba.violating_precicts(df, True))
print('Top 5 Issuer precicts')
print(pba.issuing_precincts(df, True))
print('Violation frequencies from top 3 issuer precicts')
print(pba.violation_code_frequency_top3_precincts(df, True))

Top 5 Violation precicts
+------------------+-------------------------+
|violation_precinct|count(violation_precinct)|
+------------------+-------------------------+
|                 0|                    44347|
|                19|                     3027|
|                18|                     2458|
|                14|                     2389|
|                13|                     2204|
+------------------+-------------------------+
only showing top 5 rows

[Row(violation_precinct='0', count(violation_precinct)=44347), Row(violation_precinct='19', count(violation_precinct)=3027), Row(violation_precinct='18', count(violation_precinct)=2458), Row(violation_precinct='14', count(violation_precinct)=2389), Row(violation_precinct='13', count(violation_precinct)=2204)]
Top 5 Issuer precicts
+---------------+----------------------+
|issuer_precinct|count(issuer_precinct)|
+---------------+----------------------+
|              0|                 47152|
|             19|             

In [69]:
# spark.stop() #TODO: Use this at the end of the script